In [1]:
#import sima and other dependents

import sima
import sima.motion
from sima.motion import HiddenMarkov2D
import numpy as np
import os

In [2]:
filename = 'VJ_OFC_mThalaxons_06_310_Poisson12s'
filename = 'itp_lhganiii_bl3_935_0001'

folder = '/media/stuberlab/Vijay5/2p data/VJ_OFC_mThalaxons_06/D1'
folder = 'C:/2pData/Ivan/itp_lhganiii_bl3_678/'

In [3]:
# splices file and file directory into a single path for loading

datafile = os.path.join(folder, '%s.tif'%filename)
display(datafile)

'C:/2pData/Ivan/itp_lhganiii_bl3_678/itp_lhganiii_bl3_935_0001.tif'

In [4]:
# object that contains record of whole dataset; data not stored into memory all at once

#sequences = [sima.Sequence.create('HDF5', datafile, 'tyx')]
# datafile = os.path.join(folder, '%s.tif'%filename)
sequences = [sima.Sequence.create('TIFF', datafile)]


In [5]:
# define motion correction method
mc_approach = sima.motion.HiddenMarkov2D(granularity='row', max_displacement=[30, 50], verbose=True)

In [9]:
%%time
# apply motion correction to data
dataset = mc_approach.correct(sequences, os.path.join(folder, filename + '_mc.sima'))

Estimating model parameters.
Estimating displacements for cycle  0
Wall time: 42min 24s


In [16]:
type(dataset)

sima.imaging.ImagingDataset

In [15]:
# Save TIFF files with the time average/projection of each channel

# dataset.export_frames([[[os.path.join(folder, filename + '_mc.tif')]]], fmt='TIFF16')
dataset.export_averages([os.path.join(folder, filename + '_mc_std.tif')], projection_type='std')

In [ ]:
# 

import sima
import numpy as np
from sima.ROI import ROIList
import os
import pickle

indir = '/media/stuberlab/Vijay5/2p data/VJ_OFC_mThalaxons_06/D1'
tempfiles = os.walk(indir).next()[2]
h5files = [f for f in tempfiles if os.path.splitext(f)[1]=='.h5' and 'spatialweights' not in f]
if len(h5files) > 1:
    raise Exception('Too many .h5 files found. Only keep the data file for this session')
filename = h5files[0]
filename = os.path.splitext(filename)[0]

rois = ROIList.load(os.path.join(indir, filename + '_mc_RoiSet.zip'), fmt='ImageJ')
dataset = sima.ImagingDataset.load(os.path.join(indir, filename+'_mc.sima'))
dataset.add_ROIs(rois, 'from_ImageJ')
signals = dataset.extract(rois)
extracted_signals = np.asarray(signals['raw'])
np.save(os.path.join(indir, filename + '_extractedsignals.npy'), extracted_signals)